**Methodology is simple, Scraping the stock data of Apple or 'AAPL' from Reddit posts headlines, and then finding the movement of stock price on that date (0 if the stock data decreased, 1 if it increased or stayed the same)**

--> For scraping I simple used PRAW API to scrape the data from Reddit, and then I used the Yahoo Finance API to scrape the stock data of Apple/APPL --> Then i used pandas to save that in a CSV file, thus creating my dataset for the ML model.

In [ ]:
import praw
import pandas as pd
import re

In [ ]:
# Initialize the PRAW Reddit instance
user_agent="Scraper 1.0 by /u/Additional-Win3327/"
reddit=praw.Reddit(client_id="GBz8C6K64MJwpL1-uaKuCQ",
                   client_secret="hU-wPoSxg-q5QmJ9ckV-87sUbN7_UQ",

                   user_agent=user_agent)

In [ ]:
import praw
import pandas as pd
from datetime import datetime



# Function to scrape Reddit headlines for a stock ticker
def scrape_stock_reddit(stock_ticker, stock_name, subreddits=["stocks", "investing", "wallstreetbets"], limit=1000):
    headlines_by_date = {}

    # Refine the search query
    query = f"{stock_ticker} OR {stock_name} stock"

    for subreddit in subreddits:
        for submission in reddit.subreddit(subreddit).search(query, limit=limit):
            # Convert timestamp to date
            date = datetime.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d')
            
            # Filter relevant headlines
            if "stock" in submission.title.lower() or "shares" in submission.title.lower():
                if date not in headlines_by_date:
                    headlines_by_date[date] = []
                headlines_by_date[date].append(submission.title)
    
    # Consolidate data into a DataFrame
    data = []
    for date, headlines in headlines_by_date.items():
        data.append({
            "Date": date,
            "Headlines": " ".join(headlines)  # Merge all headlines for the date
        })
    
    return pd.DataFrame(data)

# Scrape data for Apple stock
stock_ticker = "AAPL"
stock_name = "Apple"
df = scrape_stock_reddit(stock_ticker, stock_name)

# Save the data to CSV
csv_filename = f"{stock_ticker}_reddit_stock_headlines.csv"
df.to_csv(csv_filename, index=False)
print(f"Stock-specific data saved to {csv_filename}")


Stock-specific data saved to AAPL_reddit_stock_headlines.csv


In [139]:
import yfinance as yf
import pandas as pd

# Load the previously saved CSV
csv_filename = "AAPL_reddit_stock_headlines.csv"
df = pd.read_csv(csv_filename)

# Ensure the Date column is sorted and in datetime format
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(by='Date')

# Initialize the yfinance Ticker object
ticker = yf.Ticker("AAPL")

# Fetch historical data for the stock
historical_data = ticker.history(period="max")
historical_data.reset_index(inplace=True)

# Extract relevant columns: Date, Open, Close
historical_data['Date'] = pd.to_datetime(historical_data['Date']).dt.tz_localize(None)  # Remove timezone
stock_data = historical_data[['Date', 'Open', 'Close']]

# Merge stock prices with the Reddit data
df = pd.merge(df, stock_data, on='Date', how='left')

# Add the Movement column
df['Movement'] = df.apply(
    lambda row: 1 if row['Close'] > row['Open'] else 0, axis=1
)

# Save the updated CSV
updated_csv_filename = "AAPL_reddit_stock_headlines_with_movement.csv"
df.to_csv(updated_csv_filename, index=False)
print(f"Updated CSV with movement saved to {updated_csv_filename}")



Updated CSV with movement saved to AAPL_reddit_stock_headlines_with_movement.csv


In [141]:
df_final=pd.read_csv('AAPL_reddit_stock_headlines_with_movement.csv')
df_final.head()

,Date,Headlines,Open,Close,Movement
0,2008-10-21,"AAPL Apple beats by .15, misses on revs; guide...",2.920782,2.756290,0
1,2009-03-24,Apple Stock AAPL Breaking Trend -,3.204274,3.208492,1
2,2009-04-16,"AAPL Apple Inc, Stock Message Board, News, Soc...",3.590800,3.658886,1
3,2009-05-27,Apple AAPL Stock Continues To Appreciate - Vie...,3.970096,4.008357,1
4,2011-01-18,Can Apple’s Stock Withstand the Absence of Ste...,9.927344,10.262654,1


# *Some other methods and unsuccesffull attempts to scrape data*

In [132]:
import praw
import pandas as pd
from datetime import datetime

# Initialize the PRAW Reddit instance


# Function to scrape Reddit headlines for a specific stock
def scrape_reddit(stock_name, subreddit="all", limit=1000):
    headlines_by_date = {}
    
    for submission in reddit.subreddit(subreddit).search(stock_name, limit=limit):
        # Convert timestamp to date
        date = datetime.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d')
        
        # Collect headlines
        if date not in headlines_by_date:
            headlines_by_date[date] = []
        headlines_by_date[date].append(submission.title)
    
    # Consolidate data into a DataFrame
    data = []
    for date, headlines in headlines_by_date.items():
        data.append({
            "Date": date,
            "Headlines": " ".join(headlines)  # Merge all headlines for the date
        })
    
    return pd.DataFrame(data)

# Scrape data for Apple (AAPL)
stock_name = "Apple"
df = scrape_reddit(stock_name)

# Save the data to CSV
csv_filename = f"{stock_name}_reddit_headlines.csv"
df.to_csv(csv_filename, index=False)
print(f"Data saved to {csv_filename}")


Data saved to Apple_reddit_headlines.csv


In [ ]:
headlines=[]
for post in reddit.subreddit('wallstreetbets').hot(limit=10):
    headlines.append(post.title)

In [6]:
headlines=set()
for submission in reddit.subreddit('wallstreetbets').hot(limit=5):
    headlines.add(submission.title)
    print(submission.title)
    print(submission.url)
    print(submission.score)

Daily Discussion Thread for November 28, 2024
https://www.reddit.com/r/wallstreetbets/comments/1h1tiim/daily_discussion_thread_for_november_28_2024/
75
Weekly Earnings Thread 11/25 - 11/29
https://i.redd.it/wktjsi7rxg2e1.jpeg
199
Checking my portfolio when I'm out with friends
https://v.redd.it/utax2hp72o3e1
6208
How every American this morning is picturing how it will go today talking about Stocks and Crypto
https://i.redd.it/cd7szrciin3e1.jpeg
2110
When the gambling market is closed
https://i.redd.it/8gdjqewc4o3e1.png
923


In [ ]:


valid_tickers = ['AAPL', 'TSLA', 'GOOG', 'AMZN', 'MSFT', 'NFLX']  # Example tickers
ticker_pattern = r'\b[A-Z]{1,5}\b'  # Regex for potential stock tickers (1-5 uppercase letters)

def fetch_posts_with_tickers(ticker_list, limit=100):
    """Fetch posts across Reddit with stock tickers in titles or bodies."""
    posts_with_tickers = []

    # Use Reddit's global search
    for ticker in ticker_list:
        for post in reddit.subreddit("all").search(query=ticker, limit=limit):
            # Check if the post title or body contains valid tickers
            tickers_in_post = re.findall(ticker_pattern, post.title + " " + post.selftext)
            valid_tickers_in_post = [t for t in tickers_in_post if t in ticker_list]

            if valid_tickers_in_post:
                # Fetch top comment
                post.comments.replace_more(limit=0)  # Replace "more comments" to get all comments
                top_comment_body = "No comments found."
                top_comment_score = None

                if post.comments:
                    top_comment = post.comments[0]  # Top-level highest-voted comment
                    top_comment_body = top_comment.body
                    top_comment_score = top_comment.score

                # Append data to the results
                posts_with_tickers.append({
                    'title': post.title,
                    'text': post.selftext,
                    'tickers': valid_tickers_in_post,
                    'subreddit': post.subreddit.display_name,
                    'score': post.score,
                    'comments': post.num_comments,
                    'created_at': post.created_utc,
                    'top_comment_body': top_comment_body,
                    'top_comment_score': top_comment_score
                })

    return posts_with_tickers

# Example usage

all_posts = fetch_posts_with_tickers(valid_tickers)

# Display results
for post in all_posts:
    print(f"Subreddit: {post['subreddit']}")
    print(f"Title: {post['title']}")
    print(f"Tickers: {post['tickers']}")
    print(f"Score: {post['score']}, Comments: {post['comments']}")
    print(f"Created At: {post['created_at']}")
    print(f"Top Comment: {post['top_comment_body']}")
    print(f"Top Comment Score: {post['top_comment_score']}")
    print("=" * 50)


Subreddit: stocks
Title: Are you worried about AAPL long term? 
Tickers: ['AAPL', 'AAPL']
Score: 344, Comments: 456
Created At: 1715177252.0
Top Comment: It’s just a mature company now. Slow or low growth.
Top Comment Score: 552
Subreddit: wallstreetbets
Title: AAPL; please stay up this time to keep over 7 figures
Tickers: ['AAPL']
Score: 2863, Comments: 395
Created At: 1724950083.0
Top Comment: 
**User Report**| | | |
:--|:--|:--|:--
**Total Submissions** | 10 | **First Seen In WSB** | 3 years ago
**Total Comments** | 1375 | **Previous Best DD** | 
**Account Age** | 3 years | | 

[**Join WSB Discord**](http://discord.gg/wsbverse)
Top Comment Score: 1
Subreddit: investing
Title: What's the take on AAPL's future given Trump victory, Tariffs and Elon gaining more power
Tickers: ['AAPL', 'AAPL', 'AAPL']
Score: 61, Comments: 248
Created At: 1731015652.0
Top Comment: We do not produce any phones in the US. 

Just means everyone will have to buy less phones, because everything will be more e